<a href="https://colab.research.google.com/github/innsvi/python_for_ds_task/blob/main/HW_15_2_%D0%9F%D0%B5%D1%80%D0%B5%D0%B2%D1%96%D1%80%D0%BA%D0%B0_%D1%81%D1%82%D0%B0%D1%82%D0%B8c%D1%82%D0%B8%D1%87%D0%BD%D0%B8%D1%85_%D0%B3%D1%96%D0%BF%D0%BE%D1%82%D0%B5%D0%B7_Z_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Перевірка статиcтичних гіпотез. Z-test.



0. Зчитайте дані з `data.csv` у змінну data, яка має тип pandas.DataFrame. Ми будемо далі працювати з цією змінною.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 12

In [8]:
data = pd.read_csv('/content/drive/MyDrive/data/data.csv')

In [9]:
data.head()

,Unnamed: 0,data
0,0,17.499453
1,1,19.662399
2,2,7.182823
3,3,29.841625
4,4,9.239386


Запустіть код нижче. Для коректної робити всіх подальших методів дані мають бути у вигляді одновимірного `numpy.ndarray` вектора та мати розмірність (100,). Така розмірність означає, що дані - одновимірні (колонка), якщо б розмірність була (100, 1), то дані сприймаються методами як двовимірні (таблиця), хоча для нас виглядати можуть так само.

In [10]:
data = data.data.values
data.shape

(100,)

In [12]:
type(data)


numpy.ndarray

**Завдання 1**. (7 балів)
Зчитані дані - це сума покупок в доларах наших покупців на сайті протягом липня 2025 року.

До цього місяця, ми мали продажі в середньому на чек $20.

Необхідно зʼясувати, чи вийшло у нас статистично значущо **підвищити** середній чек за липень 2025?

Аби дати відповідь - ми проведемо z-test на рівні значущості $\alpha=0.05$ двома способами. В цьому завданні ми будемо виконувати обчислення "вручну" з використанням бібліотек numpy та scipy.stats подібно до прикладу в лекції. Для цього виконайте наступні 7 кроків. Правильне виконання кожного з кроків оцінюється в 1 бал.

1. Запишіть параметри задачі у змінні Python:
    - `sample_mean` - $\bar{x}$, середнє значення в вибірці
    - `population_mean` - $\mu_0$, середнє значення в популяції (тобто те, з яким ми порівнюємо середнє вибіркове значення)
    - `population_std` - $\sigma$, вибіркове стандартне відхилення, яке ми вважаємо, що є рівним ст. відх. популяції, адже маємо достатньо велику вибірку.
    - `sample_size` - $n$, розмір вибірки
    - `alpha` - $\alpha$ рівень значущості
    
    **Увага!** Для обчислення стандартного відхилення маємо скористатись функцією `np.std(your_dataframe, ddof=1)`. Чому так - розбираємо в лекції про t-test.


2. Визначте для цієї задачі:
    - якою є гіпотеза $H_0$
    - якою є альтернативна гіпотеза $H_a$
    - з яким типом тесту ми маємо справу - лівосторонній, правосторонній чи двосторонній.

3. Обчисліть стандартну помилку SE.
4. Розрахуйте z-статистику.
5. Знайдіть критичне z-значення з допомогою бібліотеки stats.
6. Обчисліть p-value з допомогою бібліотеки stats.
7. Прийміть рішення, чи відхиляєте ви гіпотезу $H_0$. Для прийняття рішення зробіть і порівняння z-статистики з критичним значенням, і проаналізуйте p-value.

In [13]:
# 1. Запишіть параметри задачі у змінні Python:

sample_mean = np.mean(data)                 # x̄ , середнє значення в вибірці
population_mean = 20                               # 𝜇0 , середнє значення в популяції (тобто те, з яким ми порівнюємо середнє вибіркове значення)
population_std = np.std(data, ddof=1)     # 𝜎 , вибіркове стандартне відхилення, яке ми вважаємо, що є рівним ст. відх. популяції, адже маємо достатньо велику вибірку
sample_size = data.shape[0]                        # 𝑛 , розмір вибірки
alpha = 0.05                                      # рівень значущості

# подивитись значення
print("sample_mean:", sample_mean)
print("population_mean:", population_mean)
print("population_std:", population_std)
print("sample_size:", sample_size)
print("alpha:", alpha)


sample_mean: 19.37755319670441
population_mean: 20
population_std: 6.25356123240434
sample_size: 100
alpha: 0.05


2. Визначте для цієї задачі:

якою є гіпотеза  𝐻0
якою є альтернативна гіпотеза  𝐻𝑎
з яким типом тесту ми маємо справу - лівосторонній, правосторонній чи двосторонній.

ВІДПОВІДЬ на 1.2.

H0: μ = 20   (середній чек НЕ збільшився)
Ha: μ > 20   (середній чек ЗБІЛЬШИВСЯ)

Тип тесту: правосторонній (бо нас цікавить збільшення)

In [14]:
# 3. Обчисліть стандартну помилку SE.

# Формула: SE = σ / √n
standard_error = population_std / np.sqrt(sample_size)

print("=" * 60)
print("ОБЧИСЛЕННЯ СТАНДАРТНОЇ ПОМИЛКИ (SE)")
print("=" * 60)

print("Формула: SE = σ / √n")
print(f"SE = {population_std:.3f} / √{sample_size}")
print(f"SE = {population_std:.3f} / {np.sqrt(sample_size):.3f}")
print(f"SE = {standard_error:.3f}")
print()

print("Інтерпретація:")
print(
    f"В середньому, вибіркове середнє чека відхиляється "
    f"від справжнього середнього приблизно на ±{standard_error:.3f} долара."
)


ОБЧИСЛЕННЯ СТАНДАРТНОЇ ПОМИЛКИ (SE)
Формула: SE = σ / √n
SE = 6.254 / √100
SE = 6.254 / 10.000
SE = 0.625

Інтерпретація:
В середньому, вибіркове середнє чека відхиляється від справжнього середнього приблизно на ±0.625 долара.


In [15]:
# 4. Розрахуйте z-статистику.

# Формула: z = (x̄ - μ0) / SE
z_statistic = (sample_mean - population_mean) / standard_error

print("=" * 60)
print("РОЗРАХУНОК Z-СТАТИСТИКИ")
print("=" * 60)

print("Формула: z = (x̄ - μ₀) / SE")
print(f"z = ({sample_mean:.3f} - {population_mean}) / {standard_error:.3f}")
print(f"z = {(sample_mean - population_mean):.3f} / {standard_error:.3f}")
print(f"z = {z_statistic:.3f}")
print()

print("Інтерпретація:")
print(
    f"Наше вибіркове середнє чека відхиляється на {z_statistic:.3f} "
    f"стандартних помилок від гіпотетичного середнього."
)
print()

if abs(z_statistic) > 2:
    print("Це досить велике відхилення (більше 2 стандартних помилок).")
elif abs(z_statistic) > 1:
    print("Це помірне відхилення (між 1 та 2 стандартними помилками).")
else:
    print("Це невелике відхилення (менше 1 стандартної помилки).")


РОЗРАХУНОК Z-СТАТИСТИКИ
Формула: z = (x̄ - μ₀) / SE
z = (19.378 - 20) / 0.625
z = -0.622 / 0.625
z = -0.995

Інтерпретація:
Наше вибіркове середнє чека відхиляється на -0.995 стандартних помилок від гіпотетичного середнього.

Це невелике відхилення (менше 1 стандартної помилки).


In [16]:
# 5. Знайдіть критичне z-значення з допомогою бібліотеки stats.

# Для правостороннього тесту

z_critical = stats.norm.ppf(1 - alpha)

print("=" * 60)
print("КРИТИЧНЕ Z-ЗНАЧЕННЯ")
print("=" * 60)

print(f"Рівень значущості α = {alpha}")
print("Тип тесту: правосторонній (Hₐ: μ > μ₀)")
print(f"Критичне z-значення = {z_critical:.3f}")
print()

print("Інтерпретація:")
print(
    f"Якщо z-статистика > {z_critical:.3f}, "
    f"то ми відхиляємо нульову гіпотезу H₀."
)
print(
    f"Ймовірність випадково отримати z > {z_critical:.3f} "
    f"за умови, що H₀ істинна, дорівнює {alpha*100:.0f}%."
)

КРИТИЧНЕ Z-ЗНАЧЕННЯ
Рівень значущості α = 0.05
Тип тесту: правосторонній (Hₐ: μ > μ₀)
Критичне z-значення = 1.645

Інтерпретація:
Якщо z-статистика > 1.645, то ми відхиляємо нульову гіпотезу H₀.
Ймовірність випадково отримати z > 1.645 за умови, що H₀ істинна, дорівнює 5%.


In [17]:
# 6. Обчисліть p-value з допомогою бібліотеки stats.

p_value = 1 - stats.norm.cdf(z_statistic)

print("=" * 60)
print("P-VALUE")
print("=" * 60)

print(f"p-value = {p_value:.6f}")
print()

print("Інтерпретація:")
print(
    "p-value — це ймовірність випадково отримати "
    "таке або більше значення середнього чека,"
)
print(
    "якщо насправді середній чек НЕ зріс і дорівнює $20."
)
print(
    f"Ймовірність випадково отримати таке відхилення = {p_value*100:.4f}%"
)
print()

if p_value < 0.001:
    print("Це НАДЗВИЧАЙНО малоймовірно (менше 0.1%).")
elif p_value < 0.01:
    print("Це дуже малоймовірно (менше 1%).")
elif p_value < 0.05:
    print("Це малоймовірно (менше 5%).")
else:
    print("Це цілком можливо випадково.")

P-VALUE
p-value = 0.840216

Інтерпретація:
p-value — це ймовірність випадково отримати таке або більше значення середнього чека,
якщо насправді середній чек НЕ зріс і дорівнює $20.
Ймовірність випадково отримати таке відхилення = 84.0216%

Це цілком можливо випадково.


In [19]:
# 7. ПРИЙНЯТТЯ РІШЕННЯ (за методом порівняння p-value та рівня значущості)

print("\n" + "=" * 60)
print("МЕТОД 2: ПОРІВНЯННЯ p-value З РІВНЕМ ЗНАЧУЩОСТІ")
print("=" * 60)


print(f"p-value = {p_value:.6f}")
print(f"Рівень значущості α = {alpha}")
print(
    f"Порівняння: {p_value:.6f} "
    f"{'<' if p_value < alpha else '≥'} {alpha}"
)
print()

if p_value < alpha:
    print("Висновок: p-value МЕНШЕ за рівень значущості.")
    decision2 = "ВІДХИЛЯЄМО H₀"
else:
    print("Висновок: p-value НЕ менше за рівень значущості.")
    decision2 = "НЕ ВІДХИЛЯЄМО H₀"

print(f"Рішення: {decision2}")



МЕТОД 2: ПОРІВНЯННЯ p-value З РІВНЕМ ЗНАЧУЩОСТІ
p-value = 0.840216
Рівень значущості α = 0.05
Порівняння: 0.840216 ≥ 0.05

Висновок: p-value НЕ менше за рівень значущості.
Рішення: НЕ ВІДХИЛЯЄМО H₀


ФІНАЛЬНИЙ ВИСНОВОК:

На рівні значущості
𝛼 = 0.05
α=0.05 ми не маємо статистично значущих доказів, що середній чек за липень 2025 зріс порівняно з 20 доларами.

**Завдання 2.** (3 бали)
Виконайте обчислення z-test з використанням бібліотеки statsmodels.

Отримайте z-статистику та р-значення.

Виведіть p-значення та зробіть висновок, чи ми маємо достатньо доказів, аби стверджувати, що середній чек зріс.

Чи зійшлись значення z-статистику та р-значення в цьому завданні з попередніми обчисленнями?

In [20]:
from statsmodels.stats import weightstats as stests

In [21]:
# підготуємо дані

print("type(data):", type(data))
print("data.shape:", data.shape)

# Якщо data має 2 колонки (індекс + чек) — беремо другу
if data.ndim == 2 and data.shape[1] >= 2:
    sample_data = data[:, 1]
else:
    # якщо це вже 1D масив
    sample_data = data

print("sample_data.shape:", sample_data.shape)
print("Перші 5 значень:", sample_data[:5])

type(data): <class 'numpy.ndarray'>
data.shape: (100,)
sample_data.shape: (100,)
Перші 5 значень: [17.49945292 19.66239904  7.18282343 29.84162485  9.23938649]


In [25]:
#запускаємо z-тест та отримаєсо z та p-value

from statsmodels.stats.weightstats import ztest

population_mean = 20
alpha = 0.05

z_stat_sm, p_val_sm = ztest(
    data,                      # наша вибірка
    value=population_mean,     # μ0 = 20
    alternative='larger'       # Hₐ: μ > 20
)

print("="*60)
print("Z-TEST (statsmodels)")
print("="*60)
print(f"z-статистика = {z_stat_sm:.6f}")
print(f"p-value      = {p_val_sm:.6f}")
print(f"alpha        = {alpha}")

if p_val_sm < alpha:
    print("\n Висновок: ВІДХИЛЯЄМО H₀")
    print("Є достатньо доказів, що середній чек ЗРІС.")
else:
    print("\n Висновок: НЕ ВІДХИЛЯЄМО H₀")
    print("Немає достатньо доказів, що середній чек зріс.")




Z-TEST (statsmodels)
z-статистика = -0.995348
p-value      = 0.840216
alpha        = 0.05

 Висновок: НЕ ВІДХИЛЯЄМО H₀
Немає достатньо доказів, що середній чек зріс.


In [26]:
print("\n" + "="*60)
print("ВИСНОВОК (по p-value)")
print("="*60)
print(f"p-value = {p_val_sm:.6f}")
print(f"alpha   = {alpha}")

if p_val_sm < alpha:
    print("ВІДХИЛЯЄМО H₀: є докази, що середній чек ЗРІС.")
else:
    print(" НЕ ВІДХИЛЯЄМО H₀: недостатньо доказів, що середній чек зріс.")



ВИСНОВОК (по p-value)
p-value = 0.840216
alpha   = 0.05
 НЕ ВІДХИЛЯЄМО H₀: недостатньо доказів, що середній чек зріс.


In [27]:
print("\n" + "="*60)
print("ПОРІВНЯННЯ З РУЧНИМИ ОБЧИСЛЕННЯМИ")
print("="*60)

print(f"Ручний z:       {z_statistic:.6f}")
print(f"Statsmodels z:  {z_stat_sm:.6f}")
print(f"Різниця z:      {abs(z_statistic - z_stat_sm):.10f}")

print("-"*60)
print(f"Ручний p-value:      {p_value:.6f}")
print(f"Statsmodels p-value: {p_val_sm:.6f}")
print(f"Різниця p-value:     {abs(p_value - p_val_sm):.10f}")



ПОРІВНЯННЯ З РУЧНИМИ ОБЧИСЛЕННЯМИ
Ручний z:       -0.995348
Statsmodels z:  -0.995348
Різниця z:      0.0000000000
------------------------------------------------------------
Ручний p-value:      0.840216
Statsmodels p-value: 0.840216
Різниця p-value:     0.0000000000


Значення z-статистики та p-value, отримані за допомогою бібліотеки statsmodels, повністю збігаються з ручними обчисленнями. Оскільки p-value = 0.84 > α = 0.05, ми не відхиляємо нульову гіпотезу. Отже, немає достатніх статистичних доказів стверджувати, що середній чек зріс.